#Fire Data Cleaning and Preprocessing

##Imports and Data

In [88]:
import sqlite3
import requests
import zipfile
import os
import pandas as pd
from io import BytesIO
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

In [89]:
!pip install kaggle
!kaggle datasets download -d rtatman/188-million-us-wildfires
with zipfile.ZipFile('188-million-us-wildfires.zip', 'r') as zip_ref:
   zip_ref.extractall('188-million-us-wildfires')

conn = sqlite3.connect(os.path.join('188-million-us-wildfires', 'FPA_FOD_20170508.sqlite'))
#fires_og = pd.read_sql_query("SELECT * FROM Fires", conn)
fire_og = pd.read_sql_query("SELECT OBJECTID, NWCG_REPORTING_AGENCY, FIRE_YEAR, STAT_CAUSE_DESCR, DISCOVERY_DATE, FIRE_SIZE, LATITUDE, LONGITUDE, STATE, FIPS_NAME FROM Fires", conn)

Dataset URL: https://www.kaggle.com/datasets/rtatman/188-million-us-wildfires
License(s): CC0-1.0
188-million-us-wildfires.zip: Skipping, found more recently modified local copy (use --force to force download)


In [90]:
fire = fire_og.copy()

#remove unnecessary columns and rename
rename_cols = {'OBJECTID':'ID','NWCG_REPORTING_AGENCY':'Agency','FIRE_YEAR':'Year','DISCOVERY_DATE':'Discovery_Date','STAT_CAUSE_DESCR':'Cause','FIRE_SIZE':'Fire_Size','LATITUDE':'LATITUDE','LONGITUDE':'LONGITUDE','STATE':'State','FIPS_NAME':'County_Name'}
fire = fire.rename(columns=rename_cols)
fire.head()

,ID,Agency,Year,Cause,Discovery_Date,Fire_Size,LATITUDE,LONGITUDE,State,County_Name
0,1,FS,2005,Miscellaneous,2453403.5,0.10,40.036944,-121.005833,CA,Plumas
1,2,FS,2004,Lightning,2453137.5,0.25,38.933056,-120.404444,CA,Placer
2,3,FS,2004,Debris Burning,2453156.5,0.10,38.984167,-120.735556,CA,El Dorado
3,4,FS,2004,Lightning,2453184.5,0.10,38.559167,-119.913333,CA,Alpine
4,5,FS,2004,Lightning,2453184.5,0.10,38.559167,-119.933056,CA,Alpine


In [91]:
fire.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1880465 entries, 0 to 1880464
Data columns (total 10 columns):
 #   Column          Dtype  
---  ------          -----  
 0   ID              int64  
 1   Agency          object 
 2   Year            int64  
 3   Cause           object 
 4   Discovery_Date  float64
 5   Fire_Size       float64
 6   LATITUDE        float64
 7   LONGITUDE       float64
 8   State           object 
 9   County_Name     object 
dtypes: float64(4), int64(2), object(4)
memory usage: 143.5+ MB


##General Cleaning
- Change range of years to match unemployment dataset
- Convert dates to datetime
- Check for weird values


In [92]:
#filter years
fire = fire[(fire['Year'] >= 1990) & (fire['Year'] <= 2016)]

#convert to datetime
fire['Discovery_Date'] = pd.to_datetime(fire['Discovery_Date'] - 2458014.5, origin='2017-09-18', unit='D')

###Checking for weird values in rest of columns

In [93]:
print(fire["Agency"].unique())
print(fire["Cause"].unique())
print(fire["State"].unique())
print(fire["Fire_Size"].max())
print(fire["Fire_Size"].min())

['FS' 'BIA' 'TRIBE' 'BLM' 'NPS' 'BOR' 'FWS' 'ST/C&L' 'DOD' 'IA' 'DOE']
['Miscellaneous' 'Lightning' 'Debris Burning' 'Campfire' 'Equipment Use'
 'Arson' 'Children' 'Railroad' 'Smoking' 'Powerline' 'Structure'
 'Fireworks' 'Missing/Undefined']
['CA' 'NM' 'OR' 'NC' 'WY' 'CO' 'WA' 'MT' 'UT' 'AZ' 'SD' 'AR' 'NV' 'ID'
 'MN' 'TX' 'FL' 'SC' 'LA' 'OK' 'KS' 'MO' 'NE' 'MI' 'KY' 'OH' 'IN' 'VA'
 'IL' 'TN' 'GA' 'AK' 'ND' 'WV' 'WI' 'AL' 'NH' 'PA' 'MS' 'ME' 'VT' 'NY'
 'IA' 'DC' 'MD' 'CT' 'MA' 'NJ' 'HI' 'DE' 'PR' 'RI']
606945.0
1e-05


We will remove rows where state is Puerto Rico\
Note: Keeping DC for now

In [94]:
fire = fire[fire["State"]!="Puerto Rico"]

In [95]:
size_l = len(fire[fire["Fire_Size"]<1])
print("Prop rows less than 1 acre",size_l/len(fire))


Prop rows less than 1 acre 0.48502684176520167


Will remove fires less than 1 acre

In [96]:
fire = fire[fire["Fire_Size"]>1]

In [97]:
fire.info()

<class 'pandas.core.frame.DataFrame'>
Index: 746709 entries, 13 to 1880463
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   ID              746709 non-null  int64         
 1   Agency          746709 non-null  object        
 2   Year            746709 non-null  int64         
 3   Cause           746709 non-null  object        
 4   Discovery_Date  746709 non-null  datetime64[ns]
 5   Fire_Size       746709 non-null  float64       
 6   LATITUDE        746709 non-null  float64       
 7   LONGITUDE       746709 non-null  float64       
 8   State           746709 non-null  object        
 9   County_Name     497130 non-null  object        
dtypes: datetime64[ns](1), float64(3), int64(2), object(4)
memory usage: 62.7+ MB


##Handling Null Values

In [98]:
#check number of nulls for each column
print(fire.isnull().sum())

ID                     0
Agency                 0
Year                   0
Cause                  0
Discovery_Date         0
Fire_Size              0
LATITUDE               0
LONGITUDE              0
State                  0
County_Name       249579
dtype: int64


###Handling County_Name
We have latitude and longitude so we will impute the missing county names using geopandas and shapely

First we will check for weird values in LATITUDE and LONGITUDE columns

In [99]:
print('LAT min:', fire['LATITUDE'].min())
print('LON min:', fire['LONGITUDE'].min())
print('LAT max:', fire['LATITUDE'].max())
print('LON max:', fire['LONGITUDE'].max())

LAT min: 17.93972222
LON min: -178.8026
LAT max: 70.1381
LON max: -65.25694444


Import county shape file

In [100]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/us-county-boundaries.geojson'
# file_path = '/content/drive/My Drive/counties.geojson'

county_shape = gpd.read_file(file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [101]:
type(county_shape)

geopandas.geodataframe.GeoDataFrame

Import libraries and convert fire dataset to geoshape dataset where county is null

In [102]:
import geopandas as gpd
from shapely.geometry import Point

lat_lon = fire[fire["County_Name"].isnull()]
lat_lon = lat_lon[["ID","LATITUDE","LONGITUDE"]]

geo = [Point(xy) for xy in zip(lat_lon['LONGITUDE'], lat_lon['LATITUDE'])]
with_county = gpd.GeoDataFrame(lat_lon, geometry=geo)

Join datasets

In [103]:
with_county = gpd.sjoin(with_county, county_shape, how='left', predicate='intersects')
with_county.head()

<ipython-input-103-45eb94958955>:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  with_county = gpd.sjoin(with_county, county_shape, how='left', predicate='intersects')


,ID,LATITUDE,LONGITUDE,geometry,index_right,geo_point_2d,statefp,countyfp,countyns,geoid,...,csafp,cbsafp,metdivfp,funcstat,aland,awater,intptlat,intptlon,state_name,countyfp_nozero
109,110,33.718889,-117.433611,POINT (-117.43361 33.71889),893.0,"{ ""lon"": -115.99384589500001, ""lat"": 33.743680...",06,065,00277297,06065,...,348,40140,None,A,1.867121e+10,2.438832e+08,+33.7298275,-116.0022389,California,65
114,115,37.345000,-102.805833,POINT (-102.80583 37.345),703.0,"{ ""lon"": -102.560476646, ""lat"": 37.31917879910...",08,009,00198120,08009,...,None,None,None,A,6.617401e+09,6.142192e+06,+37.3097802,-102.5437412,Colorado,9
154,155,34.748333,-119.410278,POINT (-119.41028 34.74833),2798.0,"{ ""lon"": -119.126027193, ""lat"": 34.35752851689...",06,111,00277320,06111,...,348,37100,None,A,4.771988e+09,9.473454e+08,+34.3587415,-119.1331432,California,111
170,171,37.940278,-120.121667,POINT (-120.12167 37.94028),1682.0,"{ ""lon"": -119.95475358, ""lat"": 38.0276014047 }",06,109,00277319,06109,...,None,43760,None,A,5.752030e+09,1.387724e+08,+38.0214344,-119.9647335,California,109
177,178,34.466667,-119.828333,POINT (-119.82833 34.46667),1775.0,"{ ""lon"": -120.030786183, ""lat"": 34.53834408619...",06,083,00277306,06083,...,None,42200,None,A,7.084063e+09,2.729752e+09,+34.5370572,-120.0399729,California,83


In [104]:
with_county = with_county[["ID","name"]]
with_county.head()

,ID,name
109,110,Riverside
114,115,Baca
154,155,Ventura
170,171,Tuolumne
177,178,Santa Barbara


Merge datasets

In [105]:
fire = pd.merge(fire, with_county, on='ID', how='left')
fire['County_Name'] = fire['County_Name'].fillna(fire['name'])
fire.head()

,ID,Agency,Year,Cause,Discovery_Date,Fire_Size,LATITUDE,LONGITUDE,State,County_Name,name
0,14,FS,2004,Campfire,2004-09-28,6.0,38.433333,-120.510000,CA,Amador,NaN
1,17,FS,2004,Equipment Use,2004-10-06,16823.0,38.523333,-120.211667,CA,Amador,NaN
2,18,FS,2004,Equipment Use,2004-10-13,7700.0,38.780000,-120.260000,CA,El Dorado,NaN
3,25,FS,2004,Lightning,2004-07-01,2.5,33.545278,-105.229444,NM,Lincoln,NaN
4,26,FS,2004,Lightning,2004-07-20,10.0,33.315833,-105.512222,NM,Lincoln,NaN


In [106]:
fire = fire.drop(columns=["name"])
print(fire.isnull().sum())

ID                0
Agency            0
Year              0
Cause             0
Discovery_Date    0
Fire_Size         0
LATITUDE          0
LONGITUDE         0
State             0
County_Name       2
dtype: int64


In [107]:
fire = fire.dropna()

##Creating and Deleting Columns


In [108]:
fire["Month"] = fire["Discovery_Date"].dt.month.astype(int)
fire["Year"] = fire["Discovery_Date"].dt.year.astype(int)
fire = fire.drop(["ID","Discovery_Date","Agency","Cause","LATITUDE","LONGITUDE"],axis=1)
fire.head()

<ipython-input-108-81556a4a6e80>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fire["Month"] = fire["Discovery_Date"].dt.month.astype(int)
<ipython-input-108-81556a4a6e80>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fire["Year"] = fire["Discovery_Date"].dt.year.astype(int)


,Year,Fire_Size,State,County_Name,Month
0,2004,6.0,CA,Amador,9
1,2004,16823.0,CA,Amador,10
2,2004,7700.0,CA,El Dorado,10
3,2004,2.5,NM,Lincoln,7
4,2004,10.0,NM,Lincoln,7


In [109]:
fire = fire.rename(columns={"County_Name":"County"})
fire.info()

<class 'pandas.core.frame.DataFrame'>
Index: 746707 entries, 0 to 746708
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Year       746707 non-null  int64  
 1   Fire_Size  746707 non-null  float64
 2   State      746707 non-null  object 
 3   County     746707 non-null  object 
 4   Month      746707 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 34.2+ MB


##Aggregating on Month and County and Final Columns


In [110]:
fire = fire.groupby(['Month','Year', 'State','County']).agg(
    Fire_Size=('Fire_Size', 'sum'),
).reset_index()

In [111]:
fire.head()

,Month,Year,State,County,Fire_Size
0,1,1992,AL,Calhoun,8.0
1,1,1992,AL,Cherokee,5.0
2,1,1992,AL,Colbert,30.0
3,1,1992,AL,Talladega,2.0
4,1,1992,AR,Montgomery,3.0


In [112]:
fire.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205985 entries, 0 to 205984
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Month      205985 non-null  int64  
 1   Year       205985 non-null  int64  
 2   State      205985 non-null  object 
 3   County     205985 non-null  object 
 4   Fire_Size  205985 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 7.9+ MB


# Unemployment Data Preprocessing


In [113]:
import zipfile
import os
import pandas as pd

In [114]:
#Import data
!pip install kaggle
!kaggle datasets download -d jayrav13/unemployment-by-county-us
with zipfile.ZipFile('unemployment-by-county-us.zip', 'r') as zip_ref:
   zip_ref.extractall('ayrav13/unemployment-by-county-us')

Dataset URL: https://www.kaggle.com/datasets/jayrav13/unemployment-by-county-us
License(s): CC0-1.0
unemployment-by-county-us.zip: Skipping, found more recently modified local copy (use --force to force download)


In [115]:
#Load Data
og_unemploy_data = pd.read_csv("ayrav13/unemployment-by-county-us/output.csv")
ue_data_copy = og_unemploy_data.copy()
ue_data_copy.head()

,Year,Month,State,County,Rate
0,2015,February,Mississippi,Newton County,6.1
1,2015,February,Mississippi,Panola County,9.4
2,2015,February,Mississippi,Monroe County,7.9
3,2015,February,Mississippi,Hinds County,6.1
4,2015,February,Mississippi,Kemper County,10.6


In [116]:
original_rate_count = ue_data_copy['Rate'].count()
print(original_rate_count)

885548


In [117]:
#explore data
ue_data_copy.info()
ue_data_copy['Year'].max()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 885548 entries, 0 to 885547
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Year    885548 non-null  int64  
 1   Month   885548 non-null  object 
 2   State   885548 non-null  object 
 3   County  885548 non-null  object 
 4   Rate    885548 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 33.8+ MB


2016

Convert months to numerical values

In [118]:
month_mapping = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4,
    'May': 5, 'June': 6, 'July': 7, 'August': 8,
    'September': 9, 'October': 10, 'November': 11, 'December': 12
}

# numerical months
ue_data_copy['Month'] = ue_data_copy['Month'].map(month_mapping)

In [119]:
ue_data_copy.head()

,Year,Month,State,County,Rate
0,2015,2,Mississippi,Newton County,6.1
1,2015,2,Mississippi,Panola County,9.4
2,2015,2,Mississippi,Monroe County,7.9
3,2015,2,Mississippi,Hinds County,6.1
4,2015,2,Mississippi,Kemper County,10.6


Drop rows where state, county, or rate is NA

In [120]:
ue_data_copy = ue_data_copy.dropna(subset=['State', 'County', 'Rate'])
ue_data_copy.head()

,Year,Month,State,County,Rate
0,2015,2,Mississippi,Newton County,6.1
1,2015,2,Mississippi,Panola County,9.4
2,2015,2,Mississippi,Monroe County,7.9
3,2015,2,Mississippi,Hinds County,6.1
4,2015,2,Mississippi,Kemper County,10.6


In [121]:
# replace spaces and standardize the state and county columns
state_abbr = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR',
    'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE',
    'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV',
    'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY',
    'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT',
    'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV',
    'Wisconsin': 'WI', 'Wyoming': 'WY'
}

# map state names to abbreviations
ue_data_copy['State'] = ue_data_copy['State'].map(state_abbr)
ue_data_copy['County'] = ue_data_copy['County'].str.replace(' County', '', regex=False)

# Merging unemployment data with fire dataframe

Creating state-county column and combining fire and unemployment datasets

In [122]:
## create state-county column to match unemployment data
fire.head()

,Month,Year,State,County,Fire_Size
0,1,1992,AL,Calhoun,8.0
1,1,1992,AL,Cherokee,5.0
2,1,1992,AL,Colbert,30.0
3,1,1992,AL,Talladega,2.0
4,1,1992,AR,Montgomery,3.0


In [123]:
ue_data_copy.head()

,Year,Month,State,County,Rate
0,2015,2,MS,Newton,6.1
1,2015,2,MS,Panola,9.4
2,2015,2,MS,Monroe,7.9
3,2015,2,MS,Hinds,6.1
4,2015,2,MS,Kemper,10.6


In [124]:
# merge on 'Month-Year'
# use left join to keep all rows from fire and only matching rows from ue_data_copy
merged_data = fire.merge(ue_data_copy[['Month','Year', 'Rate', 'State','County']], left_on=['Year','Month', 'State','County'], right_on=['Year','Month', 'State','County'], how='left')



In [125]:
merged_data.head()


,Month,Year,State,County,Fire_Size,Rate
0,1,1992,AL,Calhoun,8.0,8.7
1,1,1992,AL,Cherokee,5.0,8.7
2,1,1992,AL,Colbert,30.0,12.1
3,1,1992,AL,Talladega,2.0,10.3
4,1,1992,AR,Montgomery,3.0,8.0


In [126]:
merged_data = merged_data.dropna(subset=['Rate'])
merged_data.head()

,Month,Year,State,County,Fire_Size,Rate
0,1,1992,AL,Calhoun,8.0,8.7
1,1,1992,AL,Cherokee,5.0,8.7
2,1,1992,AL,Colbert,30.0,12.1
3,1,1992,AL,Talladega,2.0,10.3
4,1,1992,AR,Montgomery,3.0,8.0


In [127]:
final_rate_count = merged_data['Rate'].count()
print(final_rate_count)
merged_data.info()

151277
<class 'pandas.core.frame.DataFrame'>
Index: 151277 entries, 0 to 205984
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Month      151277 non-null  int64  
 1   Year       151277 non-null  int64  
 2   State      151277 non-null  object 
 3   County     151277 non-null  object 
 4   Fire_Size  151277 non-null  float64
 5   Rate       151277 non-null  float64
dtypes: float64(2), int64(2), object(2)
memory usage: 8.1+ MB


In [129]:
total_nulls = merged_data.isnull().sum().sum()
print(total_nulls)

0


In [130]:
merged_data.reset_index(drop=True).to_csv("/content/drive/MyDrive/UnemploymentProject/CleanDatasets/unemployment_fire.csv", index=False)